[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/context_enrichment_window_around_chunk_with_llamaindex.ipynb)

# 文档检索的上下文丰富窗口

## 概述

此代码实现了向量数据库中用于文档检索的上下文丰富窗口技术。它通过向每个检索到的块添加周围的上下文来增强标准检索过程，从而提高返回信息的连贯性和完整性。

## 动机

传统的向量搜索通常返回孤立的文本块，这可能缺乏充分理解所需的必要上下文。这种方法旨在通过包含相邻的文本块来提供对检索信息的更全面的视图。

## 关键组件

1. PDF 处理和文本分块
2. 使用 FAISS 和 OpenAI 嵌入创建向量存储
3. 带有上下文窗口的自定义检索功能
4. 标准检索与上下文丰富检索的比较

## 方法详情

### 文档预处理

1. 读取 PDF 并将其转换为字符串。
2. 文本被分割成带有周围句子的块

### 向量存储创建

1. 使用 OpenAI 嵌入来创建块的向量表示。
2. 从这些嵌入创建一个 FAISS 向量存储。

### 上下文丰富的检索

LlamaIndex 有一个专门用于此类任务的解析器。[SentenceWindowNodeParser](https://docs.llamaindex.ai/en/stable/module_guides/loading/node_parsers/modules/#sentencewindownodeparser) 这个解析器将文档分割成句子。但是生成的节点包含了具有关系结构的周围句子。然后，在查询时 [MetadataReplacementPostProcessor](https://docs.llamaindex.ai/en/stable/module_guides/querying/node_postprocessors/node_postprocessors/#metadatareplacementpostprocessor) 帮助连接回这些相关的句子。

### 检索比较

该笔记本包括一个比较标准检索与上下文丰富方法的章节。

## 这种方法的好处

1. 提供更连贯和上下文更丰富的结果
2. 在减轻其返回孤立文本片段的倾向的同时，保持了向量搜索的优势
3. 允许灵活调整上下文窗口的大小

## 结论

这种上下文丰富窗口技术为提高基于向量的文档搜索系统中检索信息的质量提供了一种有前途的方法。通过提供周围的上下文，它有助于保持检索信息的连贯性和完整性，可能有助于在问答等下游任务中实现更好的理解和更准确的响应。

<div style="text-align: center;">

<img src="../images/vector-search-comparison_context_enrichment.svg" alt="context enrichment window" style="width:70%; height:auto;">
</div>

# 包安装和导入

下面的单元格安装了运行此笔记本所需的所有必要软件包。


In [ ]:
# Install required packages
!pip install faiss-cpu llama-index python-dotenv

In [ ]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceWindowNodeParser, SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.postprocessor import MetadataReplacementPostProcessor
import faiss
import os
import sys
from dotenv import load_dotenv
from pprint import pprint

# Original path append replaced for Colab compatibility

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

# Llamaindex global settings for llm and embeddings
EMBED_DIMENSION=512
Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small", dimensions=EMBED_DIMENSION)

### 读取文档

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


In [ ]:
path = "data/"
reader = SimpleDirectoryReader(input_dir=path, required_exts=['.pdf'])
documents = reader.load_data()
print(documents[0])

### 创建向量存储和检索器

In [ ]:
# Create FaisVectorStore to store embeddings
fais_index = faiss.IndexFlatL2(EMBED_DIMENSION)
vector_store = FaissVectorStore(faiss_index=fais_index)

## 摄取管道

### 使用句子分割器的摄取管道

In [ ]:
base_pipeline = IngestionPipeline(
    transformations=[SentenceSplitter()],
    vector_store=vector_store
)

base_nodes = base_pipeline.run(documents=documents)

### Ingestion Pipeline with Sentence Window

In [ ]:
node_parser = SentenceWindowNodeParser(
    # How many sentences on both sides to capture. 
    # Setting this to 3 results in 7 sentences.
    window_size=3,
    # the metadata key for to be used in MetadataReplacementPostProcessor
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence"
)

# Create a pipeline with defined document transformations and vectorstore
pipeline = IngestionPipeline(
    transformations=[node_parser],
    vector_store=vector_store,
)

windowed_nodes = pipeline.run(documents=documents)

## Querying

In [ ]:
query = "Explain the role of deforestation and fossil fuels in climate change"

### Querying *without* Metadata Replacement 

In [ ]:
# Create vector index from base nodes
base_index = VectorStoreIndex(base_nodes)

# Instantiate query engine from vector index
base_query_engine = base_index.as_query_engine(
    similarity_top_k=1,
)

# Send query to the engine to get related node(s)
base_response = base_query_engine.query(query)

print(base_response)

#### Print Metadata of the Retrieved Node

In [ ]:
pprint(base_response.source_nodes[0].node.metadata)

### Querying with Metadata Replacement
"Metadata replacement" intutively might sound a little off topic since we're working on the base sentences. But LlamaIndex stores these "before/after sentences" in the metadata data of the nodes. Therefore to build back up these windows of sentences we need Metadata replacement post processor.

In [ ]:
# Create window index from nodes created from SentenceWindowNodeParser
windowed_index = VectorStoreIndex(windowed_nodes)

# Instantiate query enine with MetadataReplacementPostProcessor
windowed_query_engine = windowed_index.as_query_engine(
    similarity_top_k=1,
    node_postprocessors=[
        MetadataReplacementPostProcessor(
            target_metadata_key="window" # `window_metadata_key` key defined in SentenceWindowNodeParser
            )
        ],
)

# Send query to the engine to get related node(s)
windowed_response = windowed_query_engine.query(query)

print(windowed_response)

#### Print Metadata of the Retrieved Node

In [ ]:
# Window and original sentence are added to the metadata
pprint(windowed_response.source_nodes[0].node.metadata)